In [1]:
import pandas as pd
import json
import requests

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_date(date):
    return pd.Timestamp(date).date()

def get_min_date(commitStats):
    return get_date(commitStats[0].get('commitDate'))

def get_max_date(commitStats):
    return get_date(commitStats[len(commitStats) - 1].get('commitDate'))

In [4]:
IP = 'http://129.125.5.170'
PORT = '8080'
COMMIT_STATS_API_URL = '/GitHubCommitStatsRest/webresources/commitstats/repo/'
REQUEST_URL = IP  + ':' + PORT + COMMIT_STATS_API_URL
REPO_ID = '6296790'

In [5]:
response = requests.get(REQUEST_URL + REPO_ID)
json_data = json.loads(response.text)

In [6]:
commitStats = json_data['commitStats']

In [7]:
#for item in commitStats:
    #print(item.get('commitDate'))

In [8]:
print(get_min_date(commitStats))
print(get_max_date(commitStats))

2012-10-21
2016-05-06
